In [1]:
import enum
import os
import functools
import numpy as np
import PIL
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
import random
import string

## Enums

In [2]:
class ImageCategory(enum.Enum):
    EASY = "easy"
    MEDIUM = "medium"
    HARD = "hard"


class ImageType(enum.Enum):
    GROUND_TRUTH = "ground_truth"
    OBSERVATION = "observation"

## Utilities

In [3]:
def show_image(image):
    plt.figure(figsize=(12, 8))
    plt.tick_params(labelbottom=False, labelleft=False)    
    plt.imshow(image, cmap='gray')


def save_image(image_data, image_id, image_type, image_category, noise_level):
    printable_noise = int(round(100 * noise_level))
    path = os.path.join(
        "binary_images",
        f"{image_data.width}x{image_data.height}_{image_category.value}_noise{printable_noise}",
        f"image_{image_id}_{image_type.value}.png",
    )
    os.makedirs(os.path.dirname(path), exist_ok=True)
    image_data.save(path, format="PNG")

## Drawing

In [4]:
def draw_ellipse(image, max_width, max_height, color=255):
    origin = (random.randint(0, image.width), random.randint(0, image.height))
    width = random.randint(int(0.2 * max_width), max_width + 1)
    height = random.randint(int(0.2 * max_height), max_height + 1)
    ImageDraw.Draw(image).ellipse(
        xy=[(origin[0] - width/2, origin[1] - height/2),
            (origin[0] + width/2, origin[1] + height/2)],
        fill=color)


def draw_circle(image, max_width, max_height, color=255):
    origin = (random.randint(0, image.width), random.randint(0, image.height))
    max_size = min(max_width, max_height)
    size = random.randint(int(0.2 * max_size), max_size + 1)
    ImageDraw.Draw(image).ellipse(
        xy=[(origin[0] - size/2, origin[1] - size/2),
            (origin[0] + size/2, origin[1] + size/2)],
        fill=color)


def draw_rectangle(image, max_width, max_height, color=255):
    origin = (random.randint(0, image.width), random.randint(0, image.height))
    width = random.randint(int(0.2 * max_width), max_width + 1)
    height = random.randint(int(0.2 * max_height), max_height + 1)    
    ImageDraw.Draw(image).rectangle(
        xy=[(origin[0] - width/2, origin[1] - height/2),
            (origin[0] + width/2, origin[1] + height/2)],
        fill=color)


def draw_square(image, max_width, max_height, color=255):
    origin = (random.randint(0, image.width), random.randint(0, image.height))
    max_size = min(max_width, max_height)
    size = random.randint(int(0.2 * max_size), max_size + 1)
    ImageDraw.Draw(image).rectangle(
        xy=[(origin[0] - size/2, origin[1] - size/2),
            (origin[0] + size/2, origin[1] + size/2)],
        fill=color)


def draw_rounded_rectangle(image, max_width, max_height, color=255):
    origin = (random.randint(0, image.width), random.randint(0, image.height))
    width = random.randint(int(0.4 * max_width), max_width + 1)
    height = random.randint(int(0.4 * max_height), max_height + 1)
    radius = random.randint(
        int(min(width, height) / 4.0), int(min(width, height) / 2.0)
    )
    ImageDraw.Draw(image).rounded_rectangle(
        xy=[(origin[0] - width/2, origin[1] - height/2),
            (origin[0] + width/2, origin[1] + height/2)],
        fill=color,
        radius=radius)


def draw_line(image, max_width, max_height, color=255):
    origin = (random.randint(0, image.width), random.randint(0, image.height))  
    width = random.randint(int(0.1 * max_width), max_width + 1)
    height = random.randint(int(0.1 * max_height), max_height + 1)
    thickness = random.randint(1, int(0.02 * min(image.width, image.height)))
    start_x, end_x =  origin[0] - width/2, origin[0] + width/2
    if np.random.randint(2):
        start_x, end_x = end_x, start_x
    start_y, end_y = origin[1] - height/2, origin[1] + height/2
    if np.random.randint(2):
        start_y, end_y = end_y, start_y    
    ImageDraw.Draw(image).line(
        xy=[(start_x, start_y), (end_x, end_y)],
        width=thickness,
        fill=color)


def draw_triangle(image, max_width, max_height, color=255):
    width = random.randint(int(0.3 * max_width), max_width + 1)
    height = random.randint(int(0.3 * max_height), max_height + 1) 
    point1 = (
        random.randint(0, int(0.8 * image.width)),
        random.randint(0, int(0.8 * image.height)),
    )
    offset = (
        random.randint(int(0.4 * width), width),
        random.randint(int(0.4 * height), height),
    )
    point2 = (point1[0] + offset[0], point1[1] + offset[1])
    point3 = (point1[0] + width, point1[1] + height)
    ImageDraw.Draw(image).polygon([point1, point2, point3], fill=color)    


def draw_text(image, max_width, max_height, color=255):
    width = random.randint(int(0.3 * max_width), max_width + 1)
    font = PIL.ImageFont.load_default()
    text_length = random.randint(1, 4)
    text = "".join([random.choice(string.ascii_letters) for _ in range(text_length)])
    image_of_text = Image.new(mode='L', size=font.getsize(text))
    ImageDraw.Draw(image_of_text).text(xy=[(0, 0)], text=text, font=font, fill=color)
    height = min(int(image_of_text.height * (width / image_of_text.width)), max_height)
    image_of_text = image_of_text.resize(size=(width, height))
    text_array = np.array(image_of_text)
    text_array[text_array > 127.5] = 255
    text_array[text_array < 127.5] = 0
    image_of_text = PIL.Image.fromarray(text_array)
    left_corner = (
        random.randint(0, int(0.8 * image.width)),
        random.randint(0, int(0.8 * image.height)),
    )    
    image.paste(image_of_text, box=left_corner)

In [5]:
AVAILABLE_DRAWING_FUNCTIONS = [
    draw_ellipse, draw_circle, draw_rectangle, draw_square,
    draw_rounded_rectangle, draw_line, draw_triangle, draw_text
]

## Generating images

In [6]:
def generate_easy_image(size_of_image):
    image = Image.new(mode='L', size=size_of_image)    
    drawing_functions = AVAILABLE_DRAWING_FUNCTIONS.copy()
    drawing_functions.remove(draw_text)
    drawing_functions.remove(draw_line)
    max_width, max_height = int(image.width / 2), int(image.height / 2)
    for _ in range(np.random.randint(3, 6)):
        draw_object = random.choice(drawing_functions)
        draw_object(image, max_width, max_height, color=255)
    return image
    

def generate_medium_image(size_of_image):
    image = Image.new(mode='L', size=size_of_image)    
    drawing_functions = AVAILABLE_DRAWING_FUNCTIONS.copy()
    objects_count = np.random.randint(8, 16)
    max_ratio = 0.25 if objects_count >= 11 else 0.33 
    max_width, max_height = int(image.width * max_ratio), int(image.height * max_ratio)
    drawing_probs = np.ones(shape=(len(drawing_functions), ), dtype=np.float32)
    drawing_probs[drawing_functions.index(draw_text)] = 3.0
    drawing_probs[drawing_functions.index(draw_line)] = 3.0    
    drawing_probs /= np.sum(drawing_probs)
    for _ in range(objects_count):
        draw_object = np.random.choice(drawing_functions, p=drawing_probs)
        draw_object(image, max_width, max_height, color=255)
    return image
    

def generate_hard_image(size_of_image):
    image = generate_medium_image(size_of_image)
    drawing_functions = [draw_circle, draw_line, draw_square]
    max_width, max_height = int(image.width * 0.1), int(image.height * 0.1)
    for _ in range(50):
        draw_object = random.choice(drawing_functions)
        draw_object(image, max_width, max_height, color=0)
    return image


def generate_from_category(category, size_of_image):
    if category == ImageCategory.EASY:
        return generate_easy_image(size_of_image)
    if category == ImageCategory.MEDIUM:
        return generate_medium_image(size_of_image)
    if category == ImageCategory.HARD:
        return generate_hard_image(size_of_image)
    else:
        raise ValueError(f"Unknown category: {category}")

## Generating noise

In [7]:
def generate_noise(image, noise_level):
    array_of_image = np.array(image)
    flipped_pixels = np.random.choice(
        a=[False, True],
        size=array_of_image.shape,
        p=[1.0 - noise_level, noise_level]
    )
    array_of_image[flipped_pixels] = 255 - array_of_image[flipped_pixels]
    return PIL.Image.fromarray(array_of_image)

## Running developed generator

In [8]:
def run_for_parameters(noise_level, category, image_size):
    for image_id in range(100):
        ground_truth_image = generate_from_category(category, image_size)
        save_image(
            ground_truth_image, image_id, ImageType.GROUND_TRUTH, category, noise_level
        )
        observation = generate_noise(ground_truth_image, noise_level)
        save_image(
            observation, image_id, ImageType.OBSERVATION, category, noise_level
        )


def run_generator():
    for noise_level in np.arange(0.01, 0.151, 0.01):
        for category in [ImageCategory.EASY, ImageCategory.MEDIUM, ImageCategory.HARD]:
            for image_size in [(100, 100), (300, 300), (500, 500), (1000, 1000)]:
                run_for_parameters(noise_level, category, image_size)

In [9]:
run_generator()